In [114]:
import numpy as np
import pandas as pd
import plotly.express as px
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

df = pd.read_excel("BakeryData_Vilnius.xlsx")
#stores = ["main street A","main street B","station A","station B"]
stores = ["main street A","station A"]
daysOfTheWeek = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df["weekday_text"] = df['date'].dt.strftime('%A')
df["date"] = pd.to_datetime(df["date"])
df.set_index('date', inplace=True)
df

,weekday,main street A,main street B,station A,station B,weekday_text
date,,,,,,
2016-05-11,3,2.23,NaN,NaN,NaN,Wednesday
2016-05-12,4,18.10,NaN,NaN,NaN,Thursday
2016-05-13,5,15.85,NaN,NaN,NaN,Friday
2016-05-14,6,14.22,NaN,NaN,NaN,Saturday
2016-05-15,7,2.58,NaN,NaN,NaN,Sunday
...,...,...,...,...,...,...
2023-05-27,6,168.05,32.34,76.97,114.30,Saturday
2023-05-28,7,44.62,32.85,80.21,91.25,Sunday
2023-05-29,1,64.11,116.84,149.75,92.56,Monday


In [115]:
begin = datetime(day = 31,month = 12,year = 2016)
pre_covid = datetime(day = 1,month = 3,year = 2021)
after_covid = datetime(day = 1,month = 3,year = 2022)
df = df[(df.index > after_covid)]

def removeOutliers(df):
    mean = df.mean()
    sd = df.std()
    return df[(df > (mean - 3*sd)) & (df < (mean + 3*sd))]

In [120]:
from statsmodels.tsa.seasonal import seasonal_decompose

analysis = df['main street A'].dropna().copy()

decompose_result_mult = seasonal_decompose(analysis, model="multiplicative")

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

#decompose_result_mult.plot()

In [124]:
data = df["main street A"].dropna()

def adf_test(timeseries):
    # Perform Dickey-Fuller test
    result = adfuller(timeseries)
    print("ADF Statistic:", result[0])
    print("p-value:", result[1])
    print("Critical Values:")
    for key, value in result[4].items():
        print(f"{key}: {value}")
    
adf_test(data)

ADF Statistic: -5.947419021332137
p-value: 2.1874453531290075e-07
Critical Values:
1%: -3.445299682487321
5%: -2.8681312035123967
10%: -2.570280872933884
